# Creating my custom object detector

why object detection:

*  want to detect multiple objects 
*  want to detect different object 
*  want to count objects


difference with classification:

*  extract features from labeled parts of images instead of whole image
*  get pictures represntative for test case

<h1>concept refresh</h1>

---


-  <h4><a href="https://colab.research.google.com/drive/1rbYvb6vDelDkz13UJoFF38WvBjpIxBWt">CONVOLUTIONAL LAYER</a></h4>
-  <h4><a href="https://colab.research.google.com/drive/1rbYvb6vDelDkz13UJoFF38WvBjpIxBWt">EVALUATION METRICS</a></h4>
- <h4><a href="https://colab.research.google.com/drive/1rbYvb6vDelDkz13UJoFF38WvBjpIxBWt">IMAGE GRADIENT VECTOR</a></h4>

<h1>which model to use?</h1>

---

<table style="width:100%">
  <tr>
    <th>name</th>
    <th>type</th>
    <th>pros</th>
    <th>cons</th>
  </tr>
  <tr>
    <td>faster cnn</td>
    <td>faster training  / more accurate</td>
    <td>slower detection / requires stronger device</td>
  </tr>
  <tr>
    <td>mobile net</td>
    <td>Single Shot Detector</td>
    <td>faster detection / works better with video</td>
    <td>slower training / less accurate</td>
  </tr>
</table>

---

<h1>file structure</h1>
<span id="e0f3" class="ki kj fa ar lx b bj ml mm r mn" data-selectable-paragraph="">
object_detection/<br>├── data/<br>           │    ├── images/<br>           │    │      └── ...<br>           │    ├── <!-- -->annotations/<br>           │    │      └── ...<br>           │    ├── train_labels/<br>           │    │      └── ...<br>           │    ├── <!-- -->test_labels/<br>           │    │      └── ...<br>           │    ├── label_map.pbtxt<br>           │    ├── test_labels.csv<br>           │    ├── train_labels.csv<br>           │    ├── test_labels.records<br>           │    └── train_labels.records<br>           │<br>           └── models/           <br>                ├── research/<br>                │      ├── training/<br>                │      │      <!-- -->└── ...<br>                │      <!-- -->├── pretrained_model/<br>                <br>                │      ├── frozen_inference_graph.pb<br>                │      └── ...<br>                └── ...</span>



#get needed packages / libraries 


In [2]:
maindirectory = input('insert the name you want to give main directory \n')

insert the name you want to give main directory 
my_detector2


In [3]:
# installing some utilities 
!pip install simple_chalk
!pip install pprint
!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk
!pip install -qq Cython contextlib2 pillow lxml matplotlib
!pip install -qq pycocotools


import pandas as pd, numpy as np, matplotlib.pyplot as plt, tensorflow.compat.v1 as tf
import csv, re, os, io, glob, shutil, urllib.request, pathlib
import tarfile, xml.etree.ElementTree as ET, cv2
from PIL import Image
from collections import namedtuple, OrderedDict
from google.colab import files
import matplotlib.image as mpimg
from google.colab.patches import cv2_imshow
from pprint import pprint
from simple_chalk import chalk, green, red, blue, yellow

# make sure we train on a GPU
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
  print('Found GPU at: {}'.format(device_name))

from google.colab import drive
drive.mount('/content/drive')
# we need tenorflow v 1.15.0, object detection API is removed from tf v 2.0+

print(red(tf.__version__))
print(blue(pathlib.Path.cwd()))

  Created wheel for simple-chalk: filename=simple_chalk-0.1.0-cp36-none-any.whl size=22161 sha256=7399e3ca08efff2bfc153b9a7209d3b76a1a5f973c52ff41e2cafb2dc236aebf
  Stored in directory: /root/.cache/pip/wheels/9c/81/ae/ad4681309e28c08419ba5c41d60ccddfc050a6d6a089f79a89
Successfully built simple-chalk
  Created wheel for pprint: filename=pprint-0.1-cp36-none-any.whl size=1250 sha256=0b590f691af1850291b403ccda81ea94ba32543b5126540e15dcc794c7f15ae5
  Stored in directory: /root/.cache/pip/wheels/42/d4/c6/16a6495aecc1bda5d5857bd036efd50617789ba9bea4a05124
Successfully built pprint
Selecting previously unselected package python-bs4.
(Reading database ... 145674 files and directories currently installed.)
Preparing to unpack .../0-python-bs4_4.6.0-1_all.deb ...
Unpacking python-bs4 (4.6.0-1) ...
Selecting previously unselected package python-pkg-resources.
Preparing to unpack .../1-python-pkg-resources_39.0.1-2_all.deb ...
Unpacking python-pkg-resources (39.0.1-2) ...
Selecting previously uns

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
1.15.0
/content


# create drive directory to work with

In [4]:
# %cd /content/drive/'My Drive'/
# %mkdir $maindirectory
%cd /content/drive/'My Drive'/$maindirectory

# if no data directory present yet 
# %mkdir data
# %cd /content/drive/'My Drive'/my_detector2/data
# %mkdir images

/content/drive/My Drive/my_detector2


In [0]:
%cd /content/drive/'My Drive'/$maindirectory

# downloads the models
!git clone --q https://github.com/tensorflow/models.git

In [6]:
# compile into python files
%cd /content/drive/'My Drive'/$maindirectory/models/research/
!protoc object_detection/protos/*.proto --python_out=.
# set env variables  
# change directory name
os.environ['PYTHONPATH'] += ':/content/drive/My Drive/my_detector/models/research/:/content/drive/My Drive/my_detector2/models/research/slim'
os.environ['PYTHONPATH'] += ':./:./slim/'

/content/drive/My Drive/my_detector2/models/research


# generate tf.record files based upon train_labels csv and image specified 



    
        


In [0]:
# TO-DO replace this with label map -- in the genrate_tfrecord.py file

# def class_text_to_int(row_label):
#   if row_label == 'banana':
#     return 1

In [71]:
# something wrong here / change the maindirectory name 

%cd /content/drive/'My Drive'/my_detector2

!python3 generate_tfrecord.py \
 --csv_input=./data/train_labels.csv \
 --output_path=./data/train.record \
 --image_dir=./data/images

!python3 generate_tfrecord.py \
--csv_input=./data/test_labels.csv \
--output_path=./data/test.record \
--image_dir=./data/images

# will generate a test.record and train.record in the data directory

/content/drive/My Drive/my_detector2

read split

W0123 15:28:22.709259 139734537508736 module_wrapper.py:139] From generate_tfrecord.py:99: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.

read split
grouped split

W0123 15:28:22.810578 139734537508736 module_wrapper.py:139] From generate_tfrecord.py:56: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

writer closed
Successfully created the TFRecords: /content/drive/My Drive/my_detector2/./data/train.record

read split

W0123 15:29:40.808195 140513495152512 module_wrapper.py:139] From generate_tfrecord.py:99: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.

read split
grouped split

W0123 15:29:40.839957 140513495152512 module_wrapper.py:139] From generate_tfrecord.py:56: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

writer closed
Successfully created the TFRecords: /content/drive/My Dr

# get the model to use / configure pipeling 

[model options](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md)

In [0]:
# Some models to train on
MODELS_CONFIG = {
    'ssd_mobilenet_v2': {
        'model_name': 'ssd_mobilenet_v2_coco_2018_03_29',
    },
    'faster_rcnn_inception_v2': {
        'model_name': 'faster_rcnn_inception_v2_coco_2018_01_28',
    },
}


# Select a model from `MODELS_CONFIG`.
selected_model = 'faster_rcnn_inception_v2'
configfilename = 'faster_rcnn_inception_v2_pets.config'

In [8]:
# the distination folder where the model will be saved
# change this if you have a different working dir
%cd /content/drive/'My Drive'/$maindirectory/models/research

# folder in which there is already a model.ckpt / frozen inference graph 
# and a pipeling.config
DEST_DIR = '/content/drive/My Drive/{0}/models/research/pretrained_model'.format(maindirectory)

# Name of the object detection model to use.
MODEL = MODELS_CONFIG[selected_model]['model_name']

#selecting the model
MODEL_FILE = MODEL + '.tar.gz'

#creating the download link for the model selected
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# if model not downloaded download now 
!pwd
if not (os.path.exists(MODEL_FILE)):
    print('here')
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

#unzipping the model and extracting its content
tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

# creating an output file to save the model while training
os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)

/content/drive/My Drive/my_detector2/models/research
/content/drive/My Drive/my_detector2/models/research
here


In [9]:
# Create a folder called training inside object_detection/model/research/
%cd /content/drive/'My Drive'/$maindirectory/models/research
%mkdir training
# this directory will hold the model.ckpts of our training 
# of which we can generate a frozen inference graph later

/content/drive/My Drive/my_detector2/models/research
mkdir: cannot create directory ‘training’: File exists


In [10]:
# optional listing of all existing configs 
!ls /content/drive/'My Drive'/$maindirectory/models/research/object_detection/samples/configs


embedded_ssd_mobilenet_v1_coco.config
facessd_mobilenet_v2_quantized_320x320_open_image_v4.config
faster_rcnn_inception_resnet_v2_atrous_coco.config
faster_rcnn_inception_resnet_v2_atrous_cosine_lr_coco.config
faster_rcnn_inception_resnet_v2_atrous_oid.config
faster_rcnn_inception_resnet_v2_atrous_oid_v4.config
faster_rcnn_inception_resnet_v2_atrous_pets.config
faster_rcnn_inception_v2_coco.config
faster_rcnn_inception_v2_pets.config
faster_rcnn_nas_coco.config
faster_rcnn_resnet101_atrous_coco.config
faster_rcnn_resnet101_ava_v2.1.config
faster_rcnn_resnet101_coco.config
faster_rcnn_resnet101_fgvc.config
faster_rcnn_resnet101_kitti.config
faster_rcnn_resnet101_pets.config
faster_rcnn_resnet101_voc07.config
faster_rcnn_resnet152_coco.config
faster_rcnn_resnet152_pets.config
faster_rcnn_resnet50_coco.config
faster_rcnn_resnet50_fgvc.config
faster_rcnn_resnet50_pets.config
mask_rcnn_inception_resnet_v2_atrous_coco.config
mask_rcnn_inception_v2_coco.config
mask_rcnn_resnet101_atrous_coco.

In [11]:
# change the type of preconfig you want to use here 

%cd /content/drive/'My Drive'/$maindirectory/models/research
!cat object_detection/samples/configs/$configfilename

/content/drive/My Drive/my_detector2/models/research


# Faster R-CNN with Inception v2, configured for Oxford-IIIT Pets Dataset.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.

model {
  faster_rcnn {
    num_classes: 6
    image_resizer {
      keep_aspect_ratio_resizer {
        min_dimension: 600
        max_dimension: 1024
      }
    }
    feature_extractor {
      type: 'faster_rcnn_inception_v2'
      first_stage_features_stride: 16
    }
    first_stage_anchor_generator {
      grid_anchor_generator {
        scales: [0.25, 0.5, 1.0, 2.0]
        aspect_ratios: [0.5, 1.0, 2.0]
        height_stride: 16
        width_stride: 16
      }
    }
    first_stage_box_predictor_conv_hyperparams {
      op: CONV
      regularizer {
        l2_regularizer {
          weight

things to change:

*  number of classes possible to detect
*  fine tune checkpoint from which to we will start which we downloaded earlier and put in the pretrained_model folder 
*  input (tf record input made earlier) and label map path 
*  num examples for eval config = #images in test directory 
*  evaluation input reader and label map (which is same as for test)
*  max evals in the evaluation
*  change writefile to configfilename 

In [12]:
# don't forget to change the directory in the paths you configure
print(red(maindirectory))
print(green(configfilename))
# path joining version for other paths
DIR = '/content/drive/My Drive/{0}/data/test'.format(maindirectory)
print(blue('# images to test on:'))
print(len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))]))

my_detector2
faster_rcnn_inception_v2_pets.config
# images to test on:
72


In [13]:
#path to the config file

%%writefile object_detection/samples/configs/faster_rcnn_inception_v2_pets.config


# Faster R-CNN with Inception v2, configured for Oxford-IIIT Pets Dataset.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.

model {
  faster_rcnn {
    num_classes: 6
    image_resizer {
      keep_aspect_ratio_resizer {
        min_dimension: 600
        max_dimension: 1024
      }
    }
    feature_extractor {
      type: 'faster_rcnn_inception_v2'
      first_stage_features_stride: 16
    }
    first_stage_anchor_generator {
      grid_anchor_generator {
        scales: [0.25, 0.5, 1.0, 2.0]
        aspect_ratios: [0.5, 1.0, 2.0]
        height_stride: 16
        width_stride: 16
      }
    }
    first_stage_box_predictor_conv_hyperparams {
      op: CONV
      regularizer {
        l2_regularizer {
          weight: 0.0
        }
      }
      initializer {
        truncated_normal_initializer {
          stddev: 0.01
        }
      }
    }
    first_stage_nms_score_threshold: 0.0
    first_stage_nms_iou_threshold: 0.7
    first_stage_max_proposals: 300
    first_stage_localization_loss_weight: 2.0
    first_stage_objectness_loss_weight: 1.0
    initial_crop_size: 14
    maxpool_kernel_size: 2
    maxpool_stride: 2
    second_stage_box_predictor {
      mask_rcnn_box_predictor {
        use_dropout: false
        dropout_keep_probability: 1.0
        fc_hyperparams {
          op: FC
          regularizer {
            l2_regularizer {
              weight: 0.0
            }
          }
          initializer {
            variance_scaling_initializer {
              factor: 1.0
              uniform: true
              mode: FAN_AVG
            }
          }
        }
      }
    }
    second_stage_post_processing {
      batch_non_max_suppression {
        score_threshold: 0.0
        iou_threshold: 0.6
        max_detections_per_class: 100
        max_total_detections: 300
      }
      score_converter: SOFTMAX
    }
    second_stage_localization_loss_weight: 2.0
    second_stage_classification_loss_weight: 1.0
  }
}

train_config: {
  batch_size: 1
  optimizer {
    momentum_optimizer: {
      learning_rate: {
        manual_step_learning_rate {
          initial_learning_rate: 0.0002
          schedule {
            step: 900000
            learning_rate: .00002
          }
          schedule {
            step: 1200000
            learning_rate: .000002
          }
        }
      }
      momentum_optimizer_value: 0.9
    }
    use_moving_average: false
  }
  gradient_clipping_by_norm: 10.0
  fine_tune_checkpoint: "/content/drive/My Drive/my_detector2/models/research/pretrained_model/model.ckpt"
  from_detection_checkpoint: true
  load_all_detection_checkpoint_vars: true
  # Note: The below line limits the training process to 200K steps, which we
  # empirically found to be sufficient enough to train the pets dataset. This
  # effectively bypasses the learning rate schedule (the learning rate will
  # never decay). Remove the below line to train indefinitely.
  num_steps: 200000
  data_augmentation_options {
    random_horizontal_flip {
    }
  }
}


train_input_reader: {
  tf_record_input_reader {
    input_path: "/content/drive/My Drive/my_detector2/data/train.record"
  }
  label_map_path: "/content/drive/My Drive/my_detector2/data/label_map.pbtxt"
}

eval_config: {
  num_examples: 72
  max_evals: 10 
}

eval_input_reader: {
  tf_record_input_reader {
    input_path: "/content/drive/My Drive/my_detector2/data/test.record"
  }
  label_map_path:  "/content/drive/My Drive/my_detector2/data/label_map.pbtxt"
  shuffle: false
  num_readers: 1
}

Overwriting object_detection/samples/configs/faster_rcnn_inception_v2_pets.config


In [14]:
!cat object_detection/samples/configs/$configfilename



# Faster R-CNN with Inception v2, configured for Oxford-IIIT Pets Dataset.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.

model {
  faster_rcnn {
    num_classes: 6
    image_resizer {
      keep_aspect_ratio_resizer {
        min_dimension: 600
        max_dimension: 1024
      }
    }
    feature_extractor {
      type: 'faster_rcnn_inception_v2'
      first_stage_features_stride: 16
    }
    first_stage_anchor_generator {
      grid_anchor_generator {
        scales: [0.25, 0.5, 1.0, 2.0]
        aspect_ratios: [0.5, 1.0, 2.0]
        height_stride: 16
        width_stride: 16
      }
    }
    first_stage_box_predictor_conv_hyperparams {
      op: CONV
      regularizer {
        l2_regularizer {
          weight: 0.0
        }
      }
      initializer {
        t

# getting it integrated with tensorboard

*  make sure you have drive synced on your pc for this 
*  in command line you can start up tensorboard and point to folder in which train / eval information is saved 

In [15]:
%cd /content/drive/'My Drive'/$maindirectory/models/research
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

/content/drive/My Drive/my_detector2/models/research
--2020-01-24 08:44:37--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.196.154.11, 52.54.249.116, 34.204.59.252, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.196.154.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.2’

ngrok-stable-linux- 100%[===================>]  13.13M  13.0MB/s    in 1.0s    

2020-01-24 08:44:39 (13.0 MB/s) - ‘ngrok-stable-linux-amd64.zip.2’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [16]:
#the logs that are created while training 
# in the training folder we made earlier
# this is where we will save checkpoints and inference graph  
LOG_DIR = "training/"
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
get_ipython().system_raw('./ngrok http 6006 &')
#The link to tensorboard.
#works after the training starts.
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"


http://0e14c415.ngrok.io


`$ pip install tensorflow==1.15.0`

`$ tensorboard --logdir=D:\drive_files/my_detector/models/research/training`


# training 

*  every few minutes a checkpoint is saved 

In [17]:
# pipeline was configured up we are pointing to the file location now 

%cd /content/drive/'My Drive'/$maindirectory/models/research/
!python3 object_detection/model_main.py \
    --pipeline_config_path=/content/drive/My\ Drive/$maindirectory/models/research/object_detection/samples/configs/$configfilename \
    --model_dir=training/

/content/drive/My Drive/my_detector2/models/research
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



W0124 08:46:48.490504 139920538380160 module_wrapper.py:139] From /content/drive/My Drive/my_detector/models/research/object_detection/utils/config_util.py:102: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0124 08:46:48.496879 139920538380160 module_wrapper.py:139] From /content/drive/My Drive/my_detector/models/research/object_detection/model_lib.py:628: The name tf.logging.warning is deprecated. Please use tf.compat.v1.logging.warning instead.

W0124 08:46:48.497099 139920538380160 model_lib.py:629] Forced number of epochs for all eval va

# exporting the model inference graph

In [18]:
# exporting our model 
%cd /content/drive/'My Drive'/$maindirectory/models/research/


#dir where the model will be saved
output_directory = './drive/My\ Drive/{0}/models/research/fine_tuned_model'.format(maindirectory)

lst = os.listdir('training')
lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
last_model = lst[steps.argmax()].replace('.meta', '')

last_model_path = os.path.join('./drive/My\ Drive/{0}/models/research'.format(maindirectory),'training', last_model)
print(last_model_path)

/content/drive/My Drive/my_detector2/models/research
./drive/My\ Drive/my_detector2/models/research/training/model.ckpt-5409


In [20]:
# command to export the model as frozen inference graph 
# we can use this frozen inference graph only for inference not for training 
# we can export saved_model.pb however which we can use to resume training 
%cd /content
!python ./drive/'My Drive'/$maindirectory/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path=./drive/My\ Drive/$maindirectory/models/research/object_detection/samples/configs/$configfilename \
    --output_directory={output_directory} \
    --trained_checkpoint_prefix={last_model_path}

/content
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



W0124 09:02:04.873927 140438761895808 module_wrapper.py:139] From ./drive/My Drive/my_detector2/models/research/object_detection/export_inference_graph.py:145: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0124 09:02:04.883853 140438761895808 module_wrapper.py:139] From /content/drive/My Drive/my_detector/models/research/object_detection/exporter.py:402: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.


W0124 09:02:04.884402 140438761895808 module_wrapper.py:139] From /content/drive/My Drive/my_detector/models/research/object_detection/exporter.py:12

# start inference 

[notebook importing the graph / making an inference](https://colab.research.google.com/drive/1yuZqWmV1FcTcaTcf4w7W16pYkBHyqsvJ)

# downloading the graph / labelmap

In [22]:
#downloads the frozen model that is needed for inference
# output_directory = 'fine_tuned_model' dir specified above.

files.download('/content/drive/My Drive/{0}/models/research/fine_tuned_model'.format(maindirectory) + '/frozen_inference_graph.pb')


----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 44586, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 320, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 351, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 364, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 724, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil

In [0]:
files.download('/content/drive/My Drive/{0}/data'.format(maindirectory) + '/label_map.pbtxt')

In [0]:
files.download('/content/drive/My Drive/{0}/models/research/fine_tuned_model/saved_model'.format(maindirectory) + '/saved_model.pb')


# non working code pieces

In [0]:
%cd /content/drive/'My Drive'/$maindirectory
#adjusted from: https://github.com/datitran/raccoon_dataset
from object_detection.utils import dataset_util


#change this to the base directory where your data/ is 
data_base_url = "/content/drive/My Drive/{}/data/images/".format(maindirectory)

#location of images
image_dir = data_base_url +'images/'

def class_text_to_int(row_label):
  if row_label == 'pistol':
    return 1
  else:
    None

def split(df, group):
  data = namedtuple('data', ['filename', 'object'])
  gb = df.groupby(group)
  return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]

def create_tf_example(group, path):
	with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
    encoded_jpg = fid.read()
  encoded_jpg_io = io.BytesIO(encoded_jpg)
  image = Image.open(encoded_jpg_io)
  width, height = image.size
  filename = group.filename.encode('utf8')
  image_format = b'jpg'
  xmins = []
  xmaxs = []
  ymins = []
  ymaxs = []
  classes_text = []
  classes = []
  for index, row in group.object.iterrows():
    xmins.append(row['xmin'] / width)
    xmaxs.append(row['xmax'] / width)
    ymins.append(row['ymin'] / height)
    ymaxs.append(row['ymax'] / height)
    classes_text.append(row['class'].encode('utf8'))
    classes.append(class_text_to_int(row['class']))
    tf_example = tf.train.Example(features=tf.train.Features(feature={
    'image/height': dataset_util.int64_feature(height),
    'image/width': dataset_util.int64_feature(width),
    'image/filename': dataset_util.bytes_feature(filename),
    'image/source_id': dataset_util.bytes_feature(filename),
    'image/encoded': dataset_util.bytes_feature(encoded_jpg),
    'image/format': dataset_util.bytes_feature(image_format),
    'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
    'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
    'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
    'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
    'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
    'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example
#creates tfrecord for both csv's

for csv in ['train_labels', 'test_labels']:
  writer = tf.io.TFRecordWriter(data_base_url + csv + '.record')
  path = os.path.join(image_dir)
  examples = pd.read_csv(data_base_url + csv + '.csv')
  grouped = split(examples, 'filename')
  for group in grouped:
    tf_example = create_tf_example(group, path)
    writer.write(tf_example.SerializeToString())
    
  writer.close()
  output_path = os.path.join(os.getcwd(), data_base_url + csv + '.record')
  print('Successfully created the TFRecords: {}'.format(data_base_url +csv + '.record'))

In [0]:
#path to the config file
%%writefile object_detection/samples/configs/ssd_mobilenet_v2_coco.config

# paste the content of the config file in the same cell here.
# SSD with Mobilenet v2 configuration for MSCOCO Dataset.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.

model {
  ssd {
    num_classes: 6
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    anchor_generator {
      ssd_anchor_generator {
        num_layers: 6
        min_scale: 0.2
        max_scale: 0.95
        aspect_ratios: 1.0
        aspect_ratios: 2.0
        aspect_ratios: 0.5
        aspect_ratios: 3.0
        aspect_ratios: 0.3333
      }
    }
    image_resizer {
      fixed_shape_resizer {
        height: 300
        width: 300
      }
    }
    box_predictor {
      convolutional_box_predictor {
        min_depth: 0
        max_depth: 0
        num_layers_before_predictor: 0
        use_dropout: false
        dropout_keep_probability: 0.8
        kernel_size: 1
        box_code_size: 4
        apply_sigmoid_to_scores: false
        conv_hyperparams {
          activation: RELU_6,
          regularizer {
            l2_regularizer {
              weight: 0.00004
            }
          }
          initializer {
            truncated_normal_initializer {
              stddev: 0.03
              mean: 0.0
            }
          }
          batch_norm {
            train: true,
            scale: true,
            center: true,
            decay: 0.9997,
            epsilon: 0.001,
          }
        }
      }
    }
    feature_extractor {
      type: 'ssd_mobilenet_v2'
      min_depth: 16
      depth_multiplier: 1.0
      conv_hyperparams {
        activation: RELU_6,
        regularizer {
          l2_regularizer {
            weight: 0.00004
          }
        }
        initializer {
          truncated_normal_initializer {
            stddev: 0.03
            mean: 0.0
          }
        }
        batch_norm {
          train: true,
          scale: true,
          center: true,
          decay: 0.9997,
          epsilon: 0.001,
        }
      }
    }
    loss {
      classification_loss {
        weighted_sigmoid {
        }
      }
      localization_loss {
        weighted_smooth_l1 {
        }
      }
      hard_example_miner {
        num_hard_examples: 3000
        iou_threshold: 0.99
        loss_type: CLASSIFICATION
        max_negatives_per_positive: 3
        min_negatives_per_image: 3
      }
      classification_weight: 1.0
      localization_weight: 1.0
    }
    normalize_loss_by_num_matches: true
    post_processing {
      batch_non_max_suppression {
        score_threshold: 1e-8
        iou_threshold: 0.6
        max_detections_per_class: 100
        max_total_detections: 100
      }
      score_converter: SIGMOID
    }
  }
}

train_config: {
  batch_size: 24
  optimizer {
    rms_prop_optimizer: {
      learning_rate: {
        exponential_decay_learning_rate {
          initial_learning_rate: 0.004
          decay_steps: 800720
          decay_factor: 0.95
        }
      }
      momentum_optimizer_value: 0.9
      decay: 0.9
      epsilon: 1.0
    }
  }
  fine_tune_checkpoint: "/content/drive/My Drive/my_detector/models/research/pretrained_model/model.ckpt"
  fine_tune_checkpoint_type:  "detection"
  # Note: The below line limits the training process to 200K steps, which we
  # empirically found to be sufficient enough to train the pets dataset. This
  # effectively bypasses the learning rate schedule (the learning rate will
  # never decay). Remove the below line to train indefinitely.
  num_steps: 10000
  data_augmentation_options {
    random_horizontal_flip {
    }
  }
  data_augmentation_options {
    ssd_random_crop {
    }
  }
}

train_input_reader: {
  tf_record_input_reader {
    input_path: "/content/drive/My Drive/my_detector/data/train.record"
  }
  label_map_path: "/content/drive/My Drive/my_detector/data/label_map.pbtxt"
}

eval_config: {
  num_examples: 32
  # Note: The below line limits the evaluation process to 10 evaluations.
  # Remove the below line to evaluate indefinitely.
  max_evals: 7
}

eval_input_reader: {
  tf_record_input_reader {
    input_path: "/content/drive/My Drive/my_detector/data/test.record"
  }
  label_map_path: "/content/drive/My Drive/my_detector/data/label_map.pbtxt"
  shuffle: false
  num_readers: 1
}